In [1]:
# #######################################################################
#  HW2 (due on 3/02/19):                                                #
# To apply Evolution Strategies (or Particle Swarm Optimization)        #
#to a neural network model for a prediction task using LSTM             #                        #
#########################################################################

########################
## CIS 600 HOMEWORK2  ##
## Anagha Fatale      ##
## SUID 438039600     ##
########################


import pandas as pd
import numpy
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.metrics import confusion_matrix,mean_squared_error
import math
from __future__ import division
# fix random seed for reproducibility
numpy.random.seed(7)
from sklearn.model_selection import train_test_split
from keras import backend as K
import numpy as np
import tensorflow as tf
from sklearn.svm import SVC  
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.utils.generic_utils import get_custom_objects
from keras import optimizers
from keras.layers import Dense, Activation, Dropout
import matplotlib.pyplot as plt 
from neupy import algorithms, environment
import copy
import random
BIG_SCORE = 1.e6
N = 30 # number of particles
STEPS = 10 # number of steps
LOSS = 'mse' # Loss function
BATCH_SIZE = 32 # Size of batches to train on

/Users/anaghafatale/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/anaghafatale/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/anaghafatale/anaconda3/lib/python3.6/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) a

In [2]:
## Below Code is taken from : https://github.com/mike-holcomb/PSOkeras/tree/master/psokeras ##

class ProgressBar:
    def __init__(self, steps, updates=10):
        self.step = 0
        self.step_size = (steps // updates)
        self.total_steps = steps
        self.updates = updates
        bar = self._make_bar(0)
        print(bar)

    def update(self, i):
        if i % self.step_size > 0:
            return
        self.step = i // self.step_size
        bar = self._make_bar(i)
        print(bar)

    def done(self):
        self.step = self.total_steps
        bar = self._make_bar(self.updates)
        print(bar)

    def _make_bar(self, x):
        bar = "["
        for x in range(self.updates):
            print("\r")
            if x < self.step:
                bar += "="
            else:
                bar = " "
        bar += "]"
        print(bar)
        return bar

In [3]:
class Particle:
    def __init__(self, model, params):
        self.model = model
        self.params = params
        self.init_weights = model.get_weights()
        self.velocities = [None] * len(self.init_weights)
        self.length = len(self.init_weights)
        for i, layer in enumerate(self.init_weights):
            self.velocities[i] = np.random.rand(*layer.shape) / 5 - 0.10
            # self.velocities[i] = np.zeros(layer.shape)

        self.best_weights = None
        self.best_score = BIG_SCORE

    def get_score(self, x, y, update=True):
        print(self.model.metrics_names)
        #5 local_score = self.model.evaluate(x, y, verbose=0)
        pred = scaler.inverse_transform(self.model.predict(x))
        # print(pred[:0])
        # Prepare Y data to also be on the original scale for interpretability.
        orig_data = scaler.inverse_transform([y])
        # print(orig_data[0])
        # Calculate RMSE.
        local_score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
        print("Local RMSE score:",local_score)
        if local_score < self.best_score and update:
            self.best_score = local_score
            self.best_weights = self.model.get_weights()

        return local_score

    def _update_velocities(self, global_best_weights, depth):
        new_velocities = [None] * len(self.init_weights)
        weights = self.model.get_weights()
        local_rand, global_rand = random.random(), random.random()

        for i, layer in enumerate(weights):
            if i >= depth:
                new_velocities[i] = self.velocities[i]
                continue
            new_v = self.params['acc'] * self.velocities[i]
            new_v = new_v + self.params['local_acc'] * local_rand * (self.best_weights[i] - layer)
            new_v = new_v + self.params['global_acc'] * global_rand * (global_best_weights[i] - layer)
            new_velocities[i] = new_v

        self.velocities = new_velocities

    def _update_weights(self, depth):
        old_weights = self.model.get_weights()
        new_weights = [None] * len(old_weights)
        for i, layer in enumerate(old_weights):
            if i>= depth:
                new_weights[i] = layer
                continue
            new_w = layer + self.velocities[i]
            new_weights[i] = new_w

        self.model.set_weights(new_weights)

    def step(self, x, y, global_best_weights,depth=None):
        if depth is None:
            depth = self.length
        self._update_velocities(global_best_weights, depth)
        self._update_weights(depth)
        return self.get_score(x, y)

    def get_best_weights(self):
        return self.best_weights

In [4]:

BIG_SCORE = 1.e6  # type: float


class Optimizer:
    def __init__(self, model, loss,
                 n=10,
                 acceleration=0.1,
                 local_rate=1.0,
                 global_rate=1.0):

        self.n_particles = n
        self.structure = model.to_json()
        self.particles = [None] * n
        self.loss = loss
        self.length = len(model.get_weights())

        params = {'acc': acceleration, 'local_acc': local_rate, 'global_acc': global_rate}

        for i in range(n-1):
            m = keras.models.model_from_json(self.structure)
            m.compile(loss=loss,optimizer='sgd')
            self.particles[i] = Particle(m, params)

        self.particles[n-1] = Particle(model, params)

        self.global_best_weights = None
        self.global_best_score = BIG_SCORE

    def fit(self, x, y, steps=0, batch_size=32):
        num_batches = x.shape[0] // batch_size

        for i, p in enumerate(self.particles):
            local_score = p.get_score(x, y)
            if local_score < self.global_best_score:
                self.global_best_score = local_score
                self.global_best_weights = p.get_best_weights()

        print("PSO -- Initial best score :")
        print(self.global_best_score)
        bar = ProgressBar(steps, updates=20)

        for i in range(steps):
            for j in range(num_batches):
                x_ = x[j*batch_size:(j+1)*batch_size,:]
                y_ = y[j*batch_size:(j+1)*batch_size]

                for p in self.particles:
                    local_score = p.step(x_, y_, self.global_best_weights)
                    #print("....",local_score)
                    if local_score < self.global_best_score:
                        self.global_best_score = local_score
                        self.global_best_weights = p.get_best_weights()

            #bar.update(i)

        bar.done()

    def get_best_model(self):
        best_model = keras.models.model_from_json(self.structure)
        best_model.set_weights(self.global_best_weights)
        best_model.compile(loss=self.loss,optimizer='sgd')
        return best_model

In [5]:
##### Below code is taken from : https://github.com/mike-holcomb/PSOkeras/blob/master/example.py ##########

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense


N = 30 # number of particles
STEPS = 10 # number of steps
LOSS = 'mse' # Loss function
BATCH_SIZE = 32 # Size of batches to train on


def build_model(loss):
    """
    Builds test Keras model for predicting Iris classifications
    :param loss (str): Type of loss - must be one of Keras accepted keras losses
    :return: Keras dense model of predefined structure
    """
    model = Sequential()
    model.add(LSTM(4, input_shape = (1, window_size)))
    model.add(Dense(1))
    model.compile(loss = 'mse', optimizer = "sgd" )
    return model


def RNN(x_train, y_train):
    """
    Runs N number of backpropagation model training simulations
    :param x_train: x values to train on
    :param y_train: target labels to train with
    :return: best model run as measured by LOSS
    """
    best_model = None
    best_score = 100.0

    print("RNN called..")
    for i in range(1):
        model_s = build_model(LOSS)
        model_s.fit(x_train, y_train,epochs = 10, batch_size = 8,verbose = 2)
        scores = model_s.evaluate(x_train, y_train, batch_size=BATCH_SIZE, verbose=0)
        pred = scaler.inverse_transform(model_s.predict(x_train))
        orig_data = scaler.inverse_transform([y_train])
        train_score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
        print("RMSE Score without pyswarm Optimization is :",train_score)
        if train_score < best_score:
            best_model = model_s
            best_score = train_score
    return best_model


if __name__ == "__main__":


# Part I: First run the recurrent neural network model
#Below code is referred from the website:
#https://www.altumintelligence.com/articles/a/Time-Series-Prediction-Using-LSTM-Deep-Neural-Networks 

    data = pd.read_csv("international-airline-passengers.csv", usecols = [1], engine = "python", skipfooter = 3)
    # Get the raw data values from the pandas data frame.
    data_raw = data.values.astype("float32")

    # We apply the MinMax scaler from sklearn
    # to normalize data in the (0, 1) interval.
    scaler = MinMaxScaler(feature_range = (0, 1))
    dataset = scaler.fit_transform(data_raw)
    TRAIN_SIZE = 0.60

    train_size = int(len(dataset) * TRAIN_SIZE)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
    print("Number of entries (training set, test set): " + str((len(train), len(test))))
    
    def create_dataset(dataset, window_size = 1):
        data_X, data_Y = [], []
        for i in range(len(dataset) - window_size - 1):
            a = dataset[i:(i + window_size), 0]
            data_X.append(a)
            data_Y.append(dataset[i + window_size, 0])
        return(np.array(data_X), np.array(data_Y))

    
    
    # Create test and training sets for one-step-ahead regression.
    window_size = 1
    x_train, y_train = create_dataset(train, window_size)
    x_test, y_test = create_dataset(test, window_size)
    print("Original training data shape:")
    print(x_train.shape)

    # Reshape the input data into appropriate form for Keras.
    x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
    x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
    print("New training data shape:")
    print(x_train.shape)

    model_s = RNN(x_train,y_train)
    
    #Part II: Then run the particle swarm optimization
    #First build model to train on (primarily used for structure, also included in swarm)
    
    model_p = build_model(LOSS)

    # Instantiate optimizer with model, loss function, and hyperparameters
    # N is Number of particles
    # acceleratin Contribution of recursive particle velocity (acceleration)
    # local_rate is contribution of locally best weights to new velocity
    # global_rate is Contribution of globally best weights to new velocity
    pso = Optimizer(model=model_p,loss=LOSS,n=N,acceleration=1.0, local_rate=0.6,global_rate=0.4)   

    # Train model on provided data
    pso.fit(x_train, y_train, steps=STEPS, batch_size=BATCH_SIZE)

    # Get a copy of the model with the globally best weights
    model_p = pso.get_best_model()
    scores = model_p.evaluate(x_train, y_train, batch_size=BATCH_SIZE, verbose=0)
        
    pred = scaler.inverse_transform(model_p.predict(x_train))
    orig_data = scaler.inverse_transform([y_train])
    
    train_score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
    print("RMSE Score with pyswarm Optimization is :",train_score)



    

Number of entries (training set, test set): (86, 58)
Original training data shape:
(84, 1)
New training data shape:
(84, 1, 1)
RNN called..
Epoch 1/10
 - 1s - loss: 0.0385
Epoch 2/10
 - 0s - loss: 0.0284
Epoch 3/10
 - 0s - loss: 0.0222
Epoch 4/10
 - 0s - loss: 0.0182
Epoch 5/10
 - 0s - loss: 0.0158
Epoch 6/10
 - 0s - loss: 0.0142
Epoch 7/10
 - 0s - loss: 0.0133
Epoch 8/10
 - 0s - loss: 0.0128
Epoch 9/10
 - 0s - loss: 0.0124
Epoch 10/10
 - 0s - loss: 0.0122
RMSE Score without pyswarm Optimization is : 56.980781835071234
['loss']
Local RMSE score: 114.22389232204237
['loss']
Local RMSE score: 100.5329877448042
['loss']
Local RMSE score: 105.72517042105231
['loss']
Local RMSE score: 104.86386582368829
['loss']
Local RMSE score: 130.33183212071663
['loss']
Local RMSE score: 109.7098171289539
['loss']
Local RMSE score: 98.35971550314439
['loss']
Local RMSE score: 129.08642106832886
['loss']
Local RMSE score: 118.80251437729201
['loss']
Local RMSE score: 104.0970357205116
['loss']
Local RMSE

['loss']
Local RMSE score: 125.90971872115277
['loss']
Local RMSE score: 43.186780677534514
['loss']
Local RMSE score: 19.454882993418707
['loss']
Local RMSE score: 93.49987464078994
['loss']
Local RMSE score: 41.040203273295546
['loss']
Local RMSE score: 21.450695520585246
['loss']
Local RMSE score: 22.827202114052426
['loss']
Local RMSE score: 26.84395614463536
['loss']
Local RMSE score: 26.236175879362694
['loss']
Local RMSE score: 37.792887367003246
['loss']
Local RMSE score: 28.516964861248372
['loss']
Local RMSE score: 70.0139945952192
['loss']
Local RMSE score: 28.042081294829682
['loss']
Local RMSE score: 119.4268537198106
['loss']
Local RMSE score: 27.183549439925915
['loss']
Local RMSE score: 29.54375058118049
['loss']
Local RMSE score: 38.99889022524901
['loss']
Local RMSE score: 44.15890087328021
['loss']
Local RMSE score: 30.539231289504887
['loss']
Local RMSE score: 16.730996989117457
['loss']
Local RMSE score: 37.60033283892269
['loss']
Local RMSE score: 32.9320846033823

['loss']
Local RMSE score: 13.77466731942973
['loss']
Local RMSE score: 60.15745496574867
['loss']
Local RMSE score: 14.117277203977416
['loss']
Local RMSE score: 27.133825049084503
['loss']
Local RMSE score: 17.692870709485522
['loss']
Local RMSE score: 33.732761304597524
['loss']
Local RMSE score: 35.08642416952157
['loss']
Local RMSE score: 14.178069620073781
['loss']
Local RMSE score: 20.512683833463132
['loss']
Local RMSE score: 18.21773641605832
['loss']
Local RMSE score: 21.72834992616149
['loss']
Local RMSE score: 133.84006087112414
['loss']
Local RMSE score: 27.829336338651885
['loss']
Local RMSE score: 13.877855173143237
['loss']
Local RMSE score: 71.80583713819466
['loss']
Local RMSE score: 14.350592840091682
['loss']
Local RMSE score: 26.74988817269732
['loss']
Local RMSE score: 28.52812419012383
['loss']
Local RMSE score: 25.97834049270068
['loss']
Local RMSE score: 19.735073126105302
['loss']
Local RMSE score: 64.86539131034354
['loss']
Local RMSE score: 20.18347836159274

['loss']
Local RMSE score: 24.44858639815137
['loss']
Local RMSE score: 28.547070782825752
['loss']
Local RMSE score: 97.0301845980694
['loss']
Local RMSE score: 21.86682863049292
['loss']
Local RMSE score: 21.6368589110239
['loss']
Local RMSE score: 26.721577785992892
['loss']
Local RMSE score: 31.39129418518077
['loss']
Local RMSE score: 25.552361506993993
['loss']
Local RMSE score: 136.09773301929175
['loss']
Local RMSE score: 21.38254285169167
['loss']
Local RMSE score: 107.34950511963916
['loss']
Local RMSE score: 21.628470600037403
['loss']
Local RMSE score: 22.110250189815815
['loss']
Local RMSE score: 100.71382216936784
['loss']
Local RMSE score: 30.082863583733413
['loss']
Local RMSE score: 20.526044278439617
['loss']
Local RMSE score: 50.68662884335176
['loss']
Local RMSE score: 75.31733952357993




















 ]
 ]
RMSE Score with pyswarm Optimization is : 22.17015593581483
